<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr2/blob/main/Yurkov%20Vladimir/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparations

Импортируем модули

In [1]:
import pandas as pd
import numpy as np
import gdown
import matplotlib.pyplot as plt
import re

Загружаем данные и преобразуем их в pandas dataframe

In [2]:
url = 'https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx'
file_name = 'data.xlsx'
gdown.download(url, file_name, quiet=False)
df_backup = pd.read_excel(file_name)
df_backup.head()

Downloading...
From: https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx
To: /content/data.xlsx
100%|██████████| 32.4M/32.4M [00:02<00:00, 13.6MB/s]


,ДокументПрохождениеМедосмотра,Клиент,КлиентДатаРождения,КлиентПол,КлиентПолПредставление,ДатаЗавершенияМедосмотра,ТипМедосмотра,ТипМедосмотраПредставление,ПсихОсвидетельствование,Контрагент,...,Оториноларингология3_Аудиометрия1_Значение,Оториноларингология3_Аудиометрия1_ЗначениеПредставление,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление,Стоматология2_ВП2_Значение,Стоматология2_ВП2_ЗначениеПредставление,Терапия2_ЗаключениеФОГК_Значение,Терапия2_ЗаключениеФОГК_ЗначениеПредставление,Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
0,7f41ca7e-7dac-11ec-80fc-0cc47aab8067,c91e3407-b81f-11e9-80cf-0cc47aab8067,1976-10-24,Женский,Женский,2022-01-25,Периодический,Периодический,False,23c173a7-30d9-11e7-80d8-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2ee1f9ac-83dd-11ec-80fc-0cc47aab8067,1a1a5fea-8b7f-11eb-80e8-0cc47aab8067,1987-11-18,Мужской,Мужской,2022-02-02,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9cbb191e-8e24-11ec-80fe-0cc47aab8067,7fe324e0-fa51-11eb-80f6-0cc47aab8067,1985-09-08,Мужской,Мужской,2022-02-15,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b8a33910-8eec-11ec-80fe-0cc47aab8067,98f9d761-8d65-11ec-80fe-0cc47aab8067,1988-04-25,Мужской,Мужской,2022-02-16,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0bb0d181-938c-11ec-80fe-0cc47aab8067,009aac7d-7bcf-11eb-80e8-0cc47aab8067,1984-04-17,Мужской,Мужской,2022-02-22,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Getting started with a dataframe

Делаем копию датафрейма

In [3]:
df = df_backup.copy()

Блок с функциями.

* `clean_clear` - функция, удаляющая дубликаты строк/столбцов и полностью пустые 
строки/столбцы.
* `regex_to_nan` - функция, находящая значения по регулярному выражению и заменяющая его на `np.nan`.
* `fills_count` - функция, возвращающая заполненность датафрейма в процентах.

In [4]:
def clean_clear(df, pflag=False):
    # Сохраняем размерность датафрейма
    shape_sum = df.shape

    # Удаляем дубликаты строк
    df = df.drop_duplicates()
    # Удаляем дубликаты столбцов
    df = df.T.drop_duplicates().T

    # Удаляем полностью пустые столбцы и строки
    df = df.dropna(axis=0, how='all')
    df = df.dropna(axis=1, how='all')

    # Выводим результаты
    if pflag:
        print(f"Форма датасета до операции: {shape_sum}")
        print(f"Форма датасета после операции: {df.shape}")

        print(f"Количество строк после удаления пустых и повторяющихся: {df.shape[0]}")
        print(f"Количество столбцов после удаления пустых и повторяющихся: {df.shape[1]}")
        print(f"\n*Удалено {shape_sum[0] - df.shape[0]} строк и {shape_sum[1] - df.shape[1]} столбцов.")
    df = df.convert_dtypes()
    return df

# Регулярное выражения для GUID
regex_GUID = "^[{]?[0-9a-fA-F]{8}" + "-([0-9a-fA-F]{4}-)" + "{3}[0-9a-fA-F]{12}[}]?$"

# Поиск и удаление GUID столбцов
def regex_to_nan(df, to_replace=regex_GUID, value=np.nan, pflag=False):
    df = df.replace(to_replace=to_replace, value=value, regex=True)
    if pflag:
        print(f"Колонки для удаления:\n{df.columns[df.isna().all()].tolist()}")
    return df

# Подсчет заполненности таблицы
def fills_count(df):
    return f"Заполненность: ~{round((1 - (df.isna().sum().sum() / df.size))*100, 2)}%"

Анализ данных.
* форма
* количество значений
* основная информация о датафрейме
* определение и подсчет типов данных до и после попытки автоматического преобразования типов `df.convert_dtypes()`
* процент заполненности данными

In [5]:
print()
print("Форма: ", df.shape)
print("Количество значений: ", df.size)
print()
display(df.info(max_cols=0))
# pd.set_option('display.max_rows', None) # (default=20) для сброса pd.reset_option('display.max_rows') | для колонок max_columns
# display(df.dtypes)
# pd.reset_option('display.max_rows')
print("\nТипы данных в фрейме:")
display(df.dtypes.value_counts())
print("\nПреобразование типов данных.")
df = df.convert_dtypes()
print("\nТипы данных после преобразования:")
display(df.dtypes.value_counts())
print()
print(fills_count(df))


Форма:  (1982, 4587)
Количество значений:  9091434

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982 entries, 0 to 1981
Columns: 4587 entries, ДокументПрохождениеМедосмотра to Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
dtypes: bool(1), datetime64[ns](43), float64(1253), object(3290)
memory usage: 69.3+ MB


None


Типы данных в фрейме:


object            3290
float64           1253
datetime64[ns]      43
bool                 1
dtype: int64


Преобразование типов данных.

Типы данных после преобразования:


string            3206
Int64             1228
boolean             85
datetime64[ns]      43
Float64             25
dtype: int64


Заполненность: ~10.83%


Поиск колонок с датами.
* нахождение по обнаруженному типу `datetime64[ns]`
* нахождение колонок, содержащих в названии слово `Дата`

Сопоставление полученных результатов между собой.
Получилось, что данные по слову `Дата` имеют действительно даты, только не определились и полностью включают в себя найденные стобцы по автоматически определенному типу данных - `datetime64[ns]`

In [6]:
datatype_list = df.columns[df.dtypes == 'datetime64[ns]'].values.tolist()
print(f"Количество столбцов с типом 'datetime64[ns]': {len(datatype_list)}")
dataword_list = df.filter(regex="Дата", axis=1).columns.values.tolist()
print(f"Количество столбцов со словом 'Дата': {len(dataword_list)}")
print()
if len(datatype_list) > len(dataword_list):
    data_diff = set(datatype_list) - set(dataword_list)
    if set(dataword_list).issubset(datatype_list):
        print("Список dataword_list входит в datatype_list\n")
elif len(datatype_list) < len(dataword_list):
    data_diff =  set(dataword_list) - set(datatype_list)
    if set(datatype_list).issubset(dataword_list):
        print("Список datatype_list входит в dataword_list\n")
else: print("Списки равны\n")

# Для вывода списка разницы колонок убрать комментарий с строки
# print(*data_diff, sep="\n")

Количество столбцов с типом 'datetime64[ns]': 43
Количество столбцов со словом 'Дата': 111

Список datatype_list входит в dataword_list



Анализ и проверка целевой колонки `ЗаключениеМК` на предмет "мусорных" данных и пропусков.

In [7]:
display(df['ЗаключениеМК'].unique().tolist())

['Годен',
 'ГоденСКоррекциейЗрения',
 'ГоденБезРаботНаВысотах',
 'НуждаетсяВДообследованииИЛечении',
 'ВременноНегоден',
 'ОграниченноГоден',
 <NA>,
 'машинист бульдозера',
 'Газоспасатель',
 ' Газоспасатель']

Создание маски с соответствующими критериям данными для целевой колонки.

In [8]:
mask_list = ['Годен', 'ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах',
       'НуждаетсяВДообследованииИЛечении', 'ВременноНегоден',
       'ОграниченноГоден']

Чистка от мусорных, дублирующих, не имеющих ценности и пустых данных.

Сначала производится чистка по маске из целевой колонки `ЗаключениеМК`, затем удаляются данные с датами, затем данные с GUID, дублирующие и пустые.

Сравнивается форма датафрейма до и после чистки.

In [9]:
print(f"Форма датафрейма до: {df.shape}")
try:
    print("Чистка от некорректных данных в целевой колонке")
    df = df[df['ЗаключениеМК'].isin(mask_list)]
    print("Чистка от колонок с датами")
    df = df.drop(dataword_list, axis='columns')
    print("Чистка от GUID колонок, полностью пустых строк и колонок, дубликатов строк и колонок...")
    df = regex_to_nan(df)
    df = clean_clear(df)
except Exception as e:
    print(e)
print(f"Форма датафрейма после: {df.shape}")

Форма датафрейма до: (1982, 4587)
Чистка от некорректных данных в целевой колонке
Чистка от колонок с датами
Чистка от GUID колонок, полностью пустых строк и колонок, дубликатов строк и колонок...
Форма датафрейма после: (1446, 1580)


#Categorization

In [10]:
print(fills_count(df))
display(df.info())

Заполненность: ~16.37%
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1446 entries, 0 to 1981
Columns: 1580 entries, КлиентПол to Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение
dtypes: boolean(40), object(151), string(1389)
memory usage: 17.1+ MB


None